<a href="https://colab.research.google.com/github/Andrew11Morozov/danya/blob/main/test_dania1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тестовое задание от @DanyaTsigl (https://t.me/DanyaTsigl).

Необходимо сделать на основе данных визуализацию в Google Data Studio. Например, построить график средней посещаемости по дням, или любые другие графики.




 Необходимо попробовать найти интересные закономерности в данных и показать их.

 ## Подготовим данные для исследования

In [1]:
import pandas as pd
from google.colab import drive
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as mth
import plotly.express as px

In [2]:
drive.mount('/content/drive/')
%cd /content/drive/My Drive/

Mounted at /content/drive/
/content/drive/My Drive


In [3]:
df = pd.read_csv('analist_test.csv', sep=';')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   event_id       320 non-null    int64 
 1   event_date     320 non-null    object
 2   customer_id    320 non-null    int64 
 3   is_attend      320 non-null    int64 
 4   group_ids      320 non-null    int64 
 5   teacher_ids    320 non-null    int64 
 6   attendance_id  320 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 17.6+ KB


In [5]:
"""###
Легенда полей
Поле	        Описание
event_id	    идентификатор урока
event_date	  дата урока
customer_id	  идентификатор ученика
is_attend	    1 — посетил, 0 — пропустил
group_ids	    идентификатор группы
teacher_ids	  идентификатор преподавателя
attendance_id	идентификатор строки
### """

'###\nЛегенда полей\nПоле\t        Описание\nevent_id\t    идентификатор урока\nevent_date\t  дата урока\ncustomer_id\t  идентификатор ученика\nis_attend\t    1 — посетил, 0 — пропустил\ngroup_ids\t    идентификатор группы\nteacher_ids\t  идентификатор преподавателя\nattendance_id\tидентификатор строки\n### '

***Описание данных из задания***


Поле	        Описание\
event_id	    - идентификатор урока\
event_date	  - дата урока\
customer_id	  - идентификатор ученика\
is_attend	    - 1 — посетил, 0 — пропустил\
group_ids	    - идентификатор группы\
teacher_ids	  - идентификатор преподавателя\
attendance_id	- идентификатор строки

In [6]:
df.isna().sum()



event_id         0
event_date       0
customer_id      0
is_attend        0
group_ids        0
teacher_ids      0
attendance_id    0
dtype: int64

***Первые наблюдения:***
- Идентификатор строки. 

Нет полного соответстввия номеру строки ее идентификатору. Строк - 320, идентификаторов - 330.
- Пропуски.

Пропусков в ланных нет.

In [7]:
df.sample(1)

,event_id,event_date,customer_id,is_attend,group_ids,teacher_ids,attendance_id
179,38,2020-06-20,12,1,1,3,190


In [8]:
# Приведем в соответствие формат даты
df['event_date'] = pd.to_datetime(df['event_date'])
#df.info() # check
#df.head() # check

In [9]:
df.sample(1)

,event_id,event_date,customer_id,is_attend,group_ids,teacher_ids,attendance_id
103,24,2020-06-10,137,0,2,6,104


In [10]:
# функция для вывода гистограммы, чтобы единообразно выводить графики.
def my_bar(df, x, y, title, y_title):
    fig = px.bar(df, x=x, y=y)
    fig.update_xaxes(tickangle=-90, tickmode = 'linear')

    fig.update_layout(title=title,
                      yaxis_title=y_title,
                      margin=dict(l=0, r=0))
    fig.show()

In [11]:
# Количество учеников на занятиях (сколько всего записалось в кружек) в наблюдаемый период по датам
t = df['event_date'].value_counts()
t = t.sort_index()

In [12]:
my_bar(t, t.index, t,
      'Количества учеников записанных в кружек по датам', 'Количество событий')

Теперь выделим тех учеников, которые посетили занятия.

In [13]:
# Количество присутствующих на занятиях по датам (при условии is_attend = 1)
t1 = df[df['is_attend'] ==1]['event_date'].value_counts()
t1 = t1.sort_index()

In [14]:
my_bar(t1, t1.index, t1,
      'Распределение количества учеников по датам', 'Количество событий')

In [15]:
# Объеденим два временных файла
t2 = pd.merge(t, t1, left_index=True, right_index=True)

In [16]:
# выделим доли пришедших на занятия от записавшихся
t2['ratio'] = t2['event_date_y']/t2['event_date_x']

In [17]:
my_bar(t2, t1.index, t2['ratio'],
      'Доли количества учеников, присутствующих на занятиях по датам', 'Доли присутстующих')

## Ответы на незаданные вопросы

Ученики каких групп более ответственны. Кто меньше всего прогуливает занятия?

In [18]:
df_groupe = pd.DataFrame([], columns=['name_groupe', 'ratio'])
#Чуть автоматизируем процесс:
for i in df['group_ids'].unique():

  # Ученики, записавшихся в  i  группу должны посетить --- занятий
  t = df[df['group_ids'] == i]
  t = t['event_date'].value_counts()
  t = t.sort_index()
  t = t.sum()
  # Ученики, записавшихся в  i  группу посетили --- занятий
  t1 = df[df['group_ids'] == i]
  t1 = t1[df['is_attend'] ==1]['event_date'].value_counts()
  t1 = t1.sort_index()
  t1 = t1.sum()
  print ('Ученики, записавшихся в группу', i, 'должны посетить', t, 'занятий.')
  print ('Ученики, записавшихся в группу', i, 'посетили       ', t1, 'занятий.')
  print ('Что составило в группе', i,'       ', round(t1/t*100,3), 'процентов.')

  df_groupe.loc[-1] = [str(i), round(t1/t*100,3)]  # adding a row
  df_groupe.index =  df_groupe.index + 1  # shifting index
  df_groupe =  df_groupe.sort_index()  # sorting by index

Ученики, записавшихся в группу 3 должны посетить 70 занятий.
Ученики, записавшихся в группу 3 посетили        61 занятий.
Что составило в группе 3         87.143 процентов.
Ученики, записавшихся в группу 1 должны посетить 68 занятий.
Ученики, записавшихся в группу 1 посетили        50 занятий.
Что составило в группе 1         73.529 процентов.
Ученики, записавшихся в группу 2 должны посетить 62 занятий.
Ученики, записавшихся в группу 2 посетили        53 занятий.
Что составило в группе 2         85.484 процентов.
Ученики, записавшихся в группу 5 должны посетить 120 занятий.
Ученики, записавшихся в группу 5 посетили        100 занятий.
Что составило в группе 5         83.333 процентов.


<ipython-input-18-65d23e53b222>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-18-65d23e53b222>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-18-65d23e53b222>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-18-65d23e53b222>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [19]:
 df_groupe = df_groupe.sort_values('ratio', ascending=False)
 df_groupe

,name_groupe,ratio
3,3,87.143
1,2,85.484
0,5,83.333
2,1,73.529


In [20]:
my_bar(df_groupe, df_groupe['name_groupe'], df_groupe['ratio'],
      'Доли посещения учениками занятий в разных группах', 'Доли присутстующих')

Особо выделяющегося раздолбайства среди групп не отмечено. Средняя посещаемость от 73 до 87 процентов в разных группах.


Может быть дело в преподавтелях? Может плохо ученики ходят к определенным преподавателям?

In [21]:
df.sample()

,event_id,event_date,customer_id,is_attend,group_ids,teacher_ids,attendance_id
232,52,2020-07-01,234,1,5,4,243


In [22]:
df_tich = pd.DataFrame([], columns=['name_tich', 'ratio'])
#Чуть автоматизируем процесс:
for i in df['teacher_ids'].unique():

  # Преподаватель --- должен провести занятия у _--- учеников
  t = df[df['teacher_ids'] == i]
  t = t['event_date'].value_counts()
  t = t.sort_index()
  t = t.sum()
  # Преподаватель --- провел занятия у _--- учеников
  t1 = df[df['teacher_ids'] == i]
  t1 = t1[df['is_attend'] ==1]['event_date'].value_counts()
  t1 = t1.sort_index()
  t1 = t1.sum()

  print ('Преподаватель', i, ' должен провести занятия у', t, ' учеников.')
  print ('Преподаватель', i, '        провести занятия у', t1, 'учеников.')
  print ('Что составило у преподавателя', i,'       ', round(t1/t*100,3), 'процентов.')

  df_tich.loc[-1] = [str(i), round(t1/t*100,3)]  # adding a row
  df_tich.index =  df_tich.index + 1  # shifting index
  df_tich = df_tich.sort_index()  # sorting by index

Преподаватель 5  должен провести занятия у 70  учеников.
Преподаватель 5         провести занятия у 61 учеников.
Что составило у преподавателя 5         87.143 процентов.
Преподаватель 3  должен провести занятия у 68  учеников.
Преподаватель 3         провести занятия у 50 учеников.
Что составило у преподавателя 3         73.529 процентов.
Преподаватель 6  должен провести занятия у 51  учеников.
Преподаватель 6         провести занятия у 45 учеников.
Что составило у преподавателя 6         88.235 процентов.
Преподаватель 4  должен провести занятия у 120  учеников.
Преподаватель 4         провести занятия у 100 учеников.
Что составило у преподавателя 4         83.333 процентов.
Преподаватель 7  должен провести занятия у 11  учеников.
Преподаватель 7         провести занятия у 8 учеников.
Что составило у преподавателя 7         72.727 процентов.


<ipython-input-22-ec46925031f8>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-22-ec46925031f8>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-22-ec46925031f8>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-22-ec46925031f8>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-22-ec46925031f8>:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [23]:
df_tich = df_tich.sort_values('ratio', ascending=False)
df_tich

,name_tich,ratio
2,6,88.235
4,5,87.143
1,4,83.333
3,3,73.529
0,7,72.727


In [24]:
my_bar(df_tich, df_tich['name_tich'], df_tich['ratio'],
      'Доли посещения учениками занятий разных преподавателей', 'Доли присутстующих')

Сильно занудных преподавателей тоже не выявлено. Посещаемость у разных преподавателей колеблеться от 88 до 72 процентов.


Еще один незаданный вопрос:\

Надо понять, почему в некоторые дни нет занятий?

In [25]:
df.sample()

,event_id,event_date,customer_id,is_attend,group_ids,teacher_ids,attendance_id
303,66,2020-07-14,593,0,1,3,314


In [26]:
df['day_of_week'] = df['event_date'].dt.dayofweek
df['name_day_of_week'] = df['event_date'].dt.day_name()

#df['weekday'] = df['Timestamp'].dt.dayofweek

In [27]:
df

,event_id,event_date,customer_id,is_attend,group_ids,teacher_ids,attendance_id,day_of_week,name_day_of_week
0,4,2020-05-21,30,1,3,5,1,3,Thursday
1,4,2020-05-21,52,1,3,5,2,3,Thursday
2,4,2020-05-21,26,1,3,5,3,3,Thursday
3,4,2020-05-21,37,1,3,5,4,3,Thursday
4,3,2020-05-21,197,1,1,3,5,3,Thursday
...,...,...,...,...,...,...,...,...,...
315,58,2020-07-15,700,1,5,4,326,2,Wednesday
316,58,2020-07-15,183,1,5,4,327,2,Wednesday
317,58,2020-07-15,654,1,5,4,328,2,Wednesday
318,58,2020-07-15,354,1,5,4,329,2,Wednesday


In [28]:
t = df.groupby(by='event_date').agg({'customer_id':'nunique'}).reset_index()
t['name_day_of_week'] = t['event_date'].dt.day_name()
t['day_of_week'] = t['event_date'].dt.dayofweek
t

,event_date,customer_id,name_day_of_week,day_of_week
0,2020-05-21,8,Thursday,3
1,2020-05-22,4,Friday,4
2,2020-05-23,4,Saturday,5
3,2020-05-25,4,Monday,0
4,2020-05-26,4,Tuesday,1
5,2020-05-27,5,Wednesday,2
6,2020-05-28,4,Thursday,3
7,2020-05-29,5,Friday,4
8,2020-05-30,4,Saturday,5
9,2020-06-01,5,Monday,0


Видимо, с 1го июля поменялось расписание: до 01-06-2020 был выходной - воскресенье, а занятия были  и по понедельникам, далее выходные воскресенье-понедельник.

А с 01-07-2020 еще уменьшилось количесто занятий до 4х дней в неделю - вторник-среда-четверг-пятница.


## Общие выводы и наблюдения:

В представленных данных логи работы кружка с дататочно постоянной  посещаемостью и ровными преподавателями. Ученики разбиты на 4 группы. Средняя посещаемость от 73 до 87 процентов в разных группах.

Преподавание в группах ведут 5 преподавателей. Посещаемость занятий у преподавтелей разная. Посещаемость у разных преподавателей колеблется от 88 до 72 процентов.

Данные логов охватывают период с 2020-05-21	(Thursday) по 2020-07-16	(Thursday). За этот период дважды поменялось расписание занятий в сторону уменьшения количества дней недели, когда занятия проводятся.
